# Модель классификации комментариев для интернет-магазина

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть, клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Значение метрики качества *F1* у модели должно быть не меньше 0.75. 

### Ход работы

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
#загрузим все необходимые библиотеки и инструменты
import torch
import transformers
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

Загрузим данные и ознакомимся с ними.

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(comments.info())
display(comments.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Выделим признаки и целевой признак - `toxic`.

In [4]:
target = comments['toxic']
features = comments['text']

Разделим датасет на обучающую и тестовую выборки.

In [5]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.5, shuffle=False)

Преобразуем данные к нужному типу.

In [6]:
corpus_train = features_train.values.astype('U')

In [7]:
corpus_test = features_test.values.astype('U')

Оставим в датасете только регулярные выражения. Воспользуемся методом `re.sub()`, применим его ко всему датасету.

In [8]:
def clear_text(text):
    text_lem = re.sub(r'[^a-zA-Z]', ' ', text)
    return text_lem

In [9]:
for i in range(len(corpus_train)):
    corpus_train[i] = clear_text(corpus_train[i])

In [10]:
for i in range(len(corpus_test)):
    corpus_test[i] = clear_text(corpus_test[i])

С помощью стемминга приведем слова к форме основы. Воспользуемся инструментом `EnglishStemmer` из библиотеки `nltk`. Применим ко всему датасету.

In [11]:
stemmer = EnglishStemmer(ignore_stopwords=False)

In [12]:
%%time
for i in range(len(corpus_train)):
    word_list = nltk.word_tokenize(corpus_train[i])
    corpus_train[i] = ' '.join([stemmer.stem(w) for w in word_list])

CPU times: user 2min 5s, sys: 175 ms, total: 2min 5s
Wall time: 2min 6s


In [13]:
%%time
for i in range(len(corpus_test)):
    word_list = nltk.word_tokenize(corpus_test[i])
    corpus_test[i] = ' '.join([stemmer.stem(w) for w in word_list])

CPU times: user 2min 4s, sys: 188 ms, total: 2min 4s
Wall time: 2min 4s


Загрузим базу стоп-слов.

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Векторизуем данные с учетом стоп-слов. Применим метод `fit` к обычающей выборке, а затем метод `transform` к обучающему и тестовому набору данных.

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(corpus_train)

In [16]:
train_x = tf_idf.transform(corpus_train)

In [17]:
test_x = tf_idf.transform(corpus_test)

Матрицы готовы, обучим на них модели.

# 2. Обучение

Обучим модели и сравним значение метрики качества F1.

### LogisticRegression

In [18]:
model = LogisticRegression(random_state=12345, solver = 'liblinear', class_weight='balanced')

In [19]:
model.fit(train_x, target_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [20]:
predicted = pd.Series(model.predict(test_x))

In [21]:
f1_score(predicted, target_test).round(2)

0.75

### Random Forest Classifier

Используем метод кросс-валидации для подбора гиперпараметров модели.

In [22]:
#исследуем работу модели с различными значениями гиперпараметра max_depth
for max_depth in range(5, 26, 5):
    rf_model = RandomForestClassifier(class_weight = 'balanced', max_depth = max_depth, n_estimators = 100)
    rf_cv = cross_val_score(rf_model, train_x, target_train, cv=3)
    print("Score при max_depth =", max_depth, ":", rf_cv)
    print("Score mean =", sum(rf_cv)/len(rf_cv))
    print()

Score при max_depth = 5 : [0.68213265 0.66091371 0.66127698]
Score mean = 0.6681077789600582

Score при max_depth = 10 : [0.69450293 0.68313593 0.70102279]
Score mean = 0.6928872158908858

Score при max_depth = 15 : [0.72792901 0.73051325 0.70030834]
Score mean = 0.7195835354932424

Score при max_depth = 20 : [0.78335088 0.76209814 0.79435211]
Score mean = 0.7799337093592739

Score при max_depth = 25 : [0.76684464 0.79560068 0.77412198]
Score mean = 0.7788557658374896



In [23]:
#исследуем работу модели с различными значениями гиперпараметра n_estimators
for estim in range(50, 201, 50):
    rf_model = RandomForestClassifier(class_weight = 'balanced', max_depth = 5, n_estimators = estim)
    rf_cv = cross_val_score(rf_model, train_x, target_train, cv=3)
    print("Score при n_estimators =", estim, ":", rf_cv)
    print("Score mean =", sum(rf_cv)/len(rf_cv))
    print()

Score при n_estimators = 50 : [0.65491051 0.6105659  0.64642401]
Score mean = 0.6373001380396385

Score при n_estimators = 100 : [0.65720409 0.66286896 0.67364819]
Score mean = 0.664573747497656

Score при n_estimators = 150 : [0.66825838 0.67542771 0.67966459]
Score mean = 0.6744502275640945

Score при n_estimators = 200 : [0.66006166 0.65414552 0.6677446 ]
Score mean = 0.6606505945095024



Для обучения модели возьмем значение гиперпараметра `max_depth` = 20 и `n_estimators` = 50. Они показали лучшее соотношение времени выполнения и качества.

In [24]:
model_1 = RandomForestClassifier(class_weight = 'balanced',
                    max_depth = 20,
                    n_estimators = 50)

In [25]:
model_1.fit(train_x, target_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=50, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [26]:
predicted_1 = pd.Series(model_1.predict(test_x))

In [27]:
f1_score(predicted_1, target_test).round(2)

0.4

# 3. Выводы

1. Значение метрики F1 достигло необходимого значения только у модели `LogisticRegression`. Модель `Random Forest Classifier` не справилась с поставленной задачей.
2. Из-за большого количества наблюдений и их сложной структуры мощности обычного компьютера периодически не хватает для полноценного анализа, из-за этого время обучения и работы возрастает, и не все модели можно детально рассмотреть.